In [ ]:
#jenna forecasting をCT 画像で置き換えてみよう。
#サーキット（細線）の上を辿って、その周りのパッチ（例えば32*32画素）をベクトルとして入力として
#出力も同じパッチを表すベクトルにする。
#入力は０〜２５５の整数値のまま要素にするか、０から１の範囲に正規化する。
#出力の各要素はそれぞれの画素が中心（現時点でのサーキットの位置）と同じクラスに属する場合は１
#属さない場合は０にする。

import keras
keras.__version__

In [ ]:
import os

csv_path = '/Users/tamurataito/Documents/datasets/jena_climate_2009_2016.csv'


f = open(csv_path)
data = f.read()
f.close()

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]

float_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]]
    float_data[i, :] = values

In [ ]:
#現時点でのサーキットの位置を中心とする周りのパッチをsamples
#そのパッチをクラス分けした結果のパッチをtargetsとして配列を返すgeneratorをつくる。
#ただし、パッチの大きさは花弁の分岐点が全ておさまる程度の大きさにする。
#パッチが画像の端に達した場合の処理も考える。
#サーキットのスタート地点を適当に選んでnumber_of_steps_on_circuitだけ進む過程での
#周りのパッチの情報をtargetsとしてかえす。

def generator(batch_size, number_of_steps_on_circuit, batch_dim):

        yield samples, targets

In [ ]:
# jenna forecastingの場合の入力は以下の通り
temp_input = next(train_gen)
temp_input[0].shape

In [ ]:
#CTセグメンテーションの場合は入力を
#(batch_size, number_of_steps_on_circuit, batch_dim)
#にする。

lookback = 1440
step = 6
delay = 144
batch_size = 128

train_gen = generator(batch_size=batch_size,
                      number_of_steps_on_circuit,
                      batch_dim)
val_gen = generator(batch_size=batch_size,
                      number_of_steps_on_circuit,
                      batch_dim)
test_gen = generator(batch_size=batch_size,
                      number_of_steps_on_circuit,
                      batch_dim)

        
# This is how many steps to draw from `val_gen`
# in order to see the whole validation set:
val_steps = (300000 - 200001 - lookback) // batch_size

# This is how many steps to draw from `test_gen`
# in order to see the whole test set:
test_steps = (len(float_data) - 300001 - lookback) // batch_size

In [ ]:
temp_input[1].shape

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import Adam

patch_dim = 32*32 #パッチをベクトルに引き伸ばした時の次元
intermediate_dim = 1024 #中間層の次元（任意）


model = Sequential()
model.add(layers.LSTM(intermediate_dim,
                     dropout=0.1,
                     recurrent_dropout=0.5,
                     return_sequences=True,
                     input_shape=(None, patch_dim)))

model.add(layers.Dense(patch_dim))

model.compile(optimizer=Adam(), loss='mae')

In [1]:
model.summary()

NameError: name 'model' is not defined

In [ ]:
history = model.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=40,
                              validation_data=val_gen,
                              validation_steps=val_steps)